# Demo with OpenAI without using third party integration

In [ ]:
import pandas as pd
from openai import OpenAI
import os
import ast
import numpy as np
import pdb

client = OpenAI

    api_key = "<API_KEY>",
)

In [ ]:
question = "What temperatus should I set my house on vacation?"

In [ ]:
response = client.chat.completions.create(
    model="gpy-3.5-turbo",messages=[{"role":"user","content":question}])

response.choices[0].message.content

# RAG implementaion with custom code to load, index, vectorize the document

In [ ]:
df = pd.read_csv('../input.csv')
df.head()

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n" ," ")
    return client.embeddings.create(input = [text], model = model).data[0].embedding



In [ ]:
get_embedding(df['text'].iloc[0])

In [ ]:
%%time 
df['text'].head(5).apply(get_embedding)

In [ ]:
%%time
df['embeddings']=df['text'].apply(get_embedding)
df.to_csv('./input-with-embedding.csv', index=False)
df.to_pickle('./input-with-embeddings.pkl')

In [ ]:
%%time 
df= pd.read_pickle('./input-with-embeddings.pkl')
df.head()

In [ ]:
question_embedding = get_embedding(question)
question, question_embedding[0:10],"..."

In [ ]:
def fn(page_embedding):
    return np.dot(page_embedding, question_embedding)
df['distance'] = df['embeddings'].apply(fn)
df.head()

In [ ]:
df.sort_values('distance', ascending=False, inplace=True)
df

In [ ]:
context = df['text'].iloc[0] + "\n" +df['text'].iloc[1] + "\n" + df['text'].iloc[2] + "\n" + df['text'].iloc[4] + "\n"
print(context)

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages = [
        {"role":"system", "content":"You are an assistant who is help answering question in polite manner"},
        {"role":"user","content": question},
        {"role":"assistant","content": f"Use this information from local data source as context to answer the user question: {context}. Please stick to this context when answering the question."}
    ])
response.choices[0].messages.content

In [ ]:
query("Netback Jobs?")

In [ ]:
query("what is Monitoring Center?")